<a href="https://colab.research.google.com/github/NileshPayghan/ML/blob/main/KerasObjMaskrcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install gdown
import gdown
url = "https://drive.google.com/uc?id=1SWVDHmcQjGykZn1mPAf7pwY7BTUCYK96"
output = "Mask Rcnn.zip"
gdown.download(url, output, quiet=False)
!unzip './Mask Rcnn.zip'

Downloading...
From: https://drive.google.com/uc?id=1SWVDHmcQjGykZn1mPAf7pwY7BTUCYK96
To: /content/Mask Rcnn.zip
100%|██████████| 480M/480M [00:02<00:00, 204MB/s]


Archive:  ./Mask Rcnn.zip
   creating: Final Mask RCNN/
   creating: Final Mask RCNN/.ipynb_checkpoints/
  inflating: Final Mask RCNN/.ipynb_checkpoints/custom-checkpoint.ipynb  
  inflating: Final Mask RCNN/custom-mask-rcnn-tf2.ipynb  
   creating: Final Mask RCNN/dataset/
   creating: Final Mask RCNN/dataset/train/
  inflating: Final Mask RCNN/dataset/train/img_000014.jpg  
  inflating: Final Mask RCNN/dataset/train/img_000015.jpg  
  inflating: Final Mask RCNN/dataset/train/img_000016.jpg  
  inflating: Final Mask RCNN/dataset/train/img_000017.jpg  
  inflating: Final Mask RCNN/dataset/train/img_000018.jpg  
  inflating: Final Mask RCNN/dataset/train/img_000019.jpg  
  inflating: Final Mask RCNN/dataset/train/img_000020.jpg  
  inflating: Final Mask RCNN/dataset/train/img_000021.jpg  
  inflating: Final Mask RCNN/dataset/train/img_000022.jpg  
  inflating: Final Mask RCNN/dataset/train/train_json.json  
   creating: Final Mask RCNN/dataset/val/
  inflating: Final Mask RCNN/dataset/v

In [ ]:
%cd Final Mask RCNN

/content/Final Mask RCNN


In [ ]:
!unzip footimages.zip

Archive:  footimages.zip
   creating: footimages/
   creating: footimages/train/
  inflating: footimages/train/5.jpg  
  inflating: footimages/train/2.jpg  
  inflating: footimages/train/1.jpg  
  inflating: footimages/train/4.jpg  
  inflating: footimages/train/3.jpg  
  inflating: footimages/train/Foot_Detector_json.json  
   creating: footimages/val/
  inflating: footimages/val/5.jpg    
  inflating: footimages/val/Foot_Detector_json.json  
  inflating: footimages/val/2.jpg    
  inflating: footimages/val/4.jpeg   
  inflating: footimages/val/1.jpg    
  inflating: footimages/val/3.jpg    


In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

In [ ]:
# Root directory of the project
ROOT_DIR = "Final Mask RCNN"

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "/content/Final Mask RCNN/mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + net and table

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 3

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

############################################################
#  Dataset
############################################################


In [ ]:
import warnings
warnings.filterwarnings('ignore')
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "foot")
        # self.add_class("object", 2, "net")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # We mostly care about the x and y coordinates of each region
        if subset == 'train':
            annotations1 = json.load(open('/content/Final Mask RCNN/footimages/train/Foot_Detector_json.json'))
        else:
            annotations1 = json.load(open('/content/Final Mask RCNN/footimages/val/Foot_Detector_json.json'))
        
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"foot": 1}

            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

def train(model): 
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("/content/Final Mask RCNN/footimages", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("/content/Final Mask RCNN/footimages", "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=0.1,
                epochs=3,
                layers='heads')
			
				
				
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)			

objects: ['foot', 'foot']
numids [1, 1]
objects: ['foot', 'foot']
numids [1, 1]
objects: ['foot']
numids [1]
objects: ['foot', 'foot']
numids [1, 1]
objects: ['foot']
numids [1]
objects: ['foot']
numids [1]
objects: ['foot', 'foot']
numids [1, 1]
objects: ['foot']
numids [1]
objects: ['foot']
numids [1]
objects: ['foot', 'foot']
numids [1, 1]
Training network heads

Starting at epoch 0. LR=0.1

Checkpoint Path: Final Mask RCNN/logs/object20221117T0710/mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')